In [1]:
import pandas as pd
import requests , json , csv , time , re
from bs4 import BeautifulSoup as bs
from google.cloud import translate
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials
subscription_key = 'e66fc219be674ca0839b810acea5a4cc'
endpoint = 'https://saym-text.cognitiveservices.azure.com/'
client = translate.Client()

In [2]:
Jobs = ['werkstudent','praktikant']
Languages = ['azure','angular','react','python','java','c++','javascript',
             'deep learning','machine learning','artificial intelligence',
             'data analysis','data science','matlab','django','front-end','back-end',
            'frontend','backend','front end','back end','linux','webpack','mysql']

In [3]:
def authenticateClient():
    credentials = CognitiveServicesCredentials(subscription_key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint, credentials=credentials)
    return text_analytics_client

def entity_recognition(Text):
    entityData = set()
    
    client = authenticateClient()

    try:
        documents = [
            {"id": "1", "language": "en", "text": Text}
        ]
        response = client.entities(documents=documents)

        for document in response.documents:
            for entity in document.entities:
                entityData.add(entity.name)
    
        response = client.key_phrases(documents=documents)

        for document in response.documents:
            for phrase in document.key_phrases:
                entityData.add(phrase)

    except Exception as err:
        print("Encountered exception. {}".format(err))
    return entityData

In [4]:
Job_search_details = pd.DataFrame(columns={'Title', 'Url','Place', 'Time'})
for p in Jobs:
    Url = 'https://www.stepstone.de/5/job-search-simple.html?stf=freeText&ns=1&ke='+p+'&ws=essen&ra=50&li=100&wt=80002'
    Request = requests.get(Url,'lxml')
    HTML = bs(Request.content)
    Links = HTML.find_all('article')
    for i in range(len(Links)):
        job_title = Links[i].h2.text
        job_links = Links[i].find_all('a')
        job_url = 'https://www.stepstone.de'+job_links[1]['href']
        job_place_time = Links[i].find_all('li')
        job_place = job_place_time[1].text
        job_time = job_place_time[0].text
        Job_search_details = Job_search_details.append({
            'Title': job_title,
            'Url' : job_url,
            'Place' : job_place,
            'Time' : job_time
        }, ignore_index = True)
#         print(job_title)

In [6]:

Temp_jobs = pd.DataFrame()
Full_Job_Des = pd.DataFrame()

for index , row in Job_search_details.iterrows():
    Job_req_des = dict()
    Title = row['Title']
    Place = row['Place']
    Time = row['Time']
    Links = row['Url']
    Request = requests.get(Links)
    Pages = bs(Request.content,'lxml')
    Job_Title = Pages.find('h1').text
    Main = Pages.find_all('main')
    if Main != None or Main != '':
        try:
            str_list = Main[1].text.split('\n')
        except:
            str_list = '[]'
            
        Texts = [x.replace('\xa0',' ') for x in str_list if x != '' and x !='\xa0' and x != ' ']
        NameSet = set()
        for Lines in Texts:
            for name in Languages:
                if Lines.lower().find(name)!=-1:
                    NameSet.add(name)
                    
        if len(NameTemp) !=0:
            En_texts = list()
            EntitySet = set()               
            Texts_en = client.translate(Texts)
            Title_en = client.translate(Job_Title)
            Title_en = Title_en['translatedText']
            
            for Eng in Texts_en:
                EngText = Eng['translatedText']
                Entites = entity_recognition(EngText)
                for i in Entites:
                    EntitySet.add(i)
                    En_texts.append(EngText)


           
            for Entity in EntitySet:
                for name in Languages:
                    if Entity.lower().find(name)!=-1:
                        NameSet.add(name)

            Full_Job_Des = Full_Job_Des.append({
                    'Job Title': Job_Title,
                    'Job Title En':Title_en,
                    'Job Url' : Links,
                    'Place' : Place,
                    'Time' : Time,
                    'Job Details': Texts,
                    'Job Details English':En_texts,
                    'Entities': EntitySet,
                    'Important Points': NameSet
                },ignore_index = True)
            print(Title_en)
Full_Job_Des.to_json('Full_Details.json')

IT working student in the field of digitization (m / f / d) up to 20 hours per week
Working student (m / f / d) digitization
Intern / Working student digital transformation (f / m / d)
Working student software development (m / f / d)
Intern / Working Student Data Analytics (f / m / d)
Working student (m / f / d) CRM test development
Working student Data Intelligence Competence Center (m / f / d)
Intern / Working student digital transformation (f / m / d)
Intern / Working Student Data Analytics (f / m / d)
Intern / Working student Predictive Analytics Solutions and Business Analytics (f / m / d)
Working student / intern (m / f / d) project engineering in the area of OEM
Reinforcement for our development team (full-time / part-time / internship)
Internship business informatics, business intelligence (m / f / d)
Working student (m / f / d) CRM test development
Working student for web development (f / m / d)
Working student (m / f / d) in the area of advanced engineering mechatronics for t

In [7]:
Full_Job_Des.to_json('Full_Details.json')

In [ ]:
# Full_Job_Des = pd.DataFrame()
# for index , row in Temp_jobs.iterrows():
#     En_texts = list()
#     EntitySet = set()        
#     Texts = row['Job Details']
#     Job_Title = row['Job Title']        
#     Texts_en = client.translate(Texts)
#     Title_en = client.translate(Job_Title)
# #     for Tit_en in Title_en:
#     Title_en = Title_en['translatedText']
    
#     for Eng in Texts_en:
#         EngText = Eng['translatedText']
#         Entites = entity_recognition(EngText)
#         for i in Entites:
#             EntitySet.add(i)
#             En_texts.append(EngText)
       
        
#     NameSet = set()
#     for Entity in EntitySet:
#         for name in Languages:
#             if Entity.lower().find(name)!=-1:
#                 NameSet.add(name)
                    
#     Full_Job_Des = Full_Job_Des.append({
#             'Job Title': Job_Title,
#             'Job Title En':Title_en,
#             'Job Url' : row['Job Url'],
#             'Place' : row['Place'],
#             'Time' : row['Time'],
#             'Job Details': Texts,
#             'Job Details English':En_texts,
#             'Entities': EntitySet,
#             'Important Points': NameSet
#         },ignore_index = True)
#     print(Title_en)
    
# Full_Job_Des.to_json('Full_Details.json')

In [8]:
Details = pd.read_json('Full_Details.json')
# for i in range(len(Details)):

#     Entities = Details['Entities'][i]
#     for j in Entities:
#         for name in Languages:
#             if j.lower().find(name.lower())!= -1:
#                 print(Details['Job Title'][i],name)
# #                 print(Entities)
# #                 print(Details['Job Details English'][i])


In [ ]:
Details